In [2]:
!pip install dlib

     |████████████████████████████████| 3.1MB 548kB/s eta 0:00:01
  ERROR: Command errored out with exit status 1:
   command: /home/thamizharasi/anaconda3/bin/python -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-qrxn2zsp/dlib/setup.py'"'"'; __file__='"'"'/tmp/pip-install-qrxn2zsp/dlib/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /tmp/pip-wheel-b9pyxsw7 --python-tag cp37
       cwd: /tmp/pip-install-qrxn2zsp/dlib/
  Complete output (53 lines):
  running bdist_wheel
  running build
  running build_py
  package init file 'dlib/__init__.py' not found (or not a regular file)
  running build_ext
  Traceback (most recent call last):
    File "/tmp/pip-install-qrxn2zsp/dlib/setup.py", line 120, in get_cmake_version
      out = subprocess.check_output(['cmake', '--version'])
    File "/home/thamizharasi/anaconda3/lib/pyt

In [3]:
from imutils import paths
import numpy as np
import argparse
import imutils
import pickle
#import dlib
#import face_recognition
import cv2
import os
import pdb
protoPath =  "./face_detection_model/deploy.prototxt"
modelPath = "./face_detection_model/res10_300x300_ssd_iter_140000.caffemodel"


print("[INFO] quantifying faces...")
imagePaths = list(paths.list_images("./dataset"))
 

# initialize the total number of faces processed
total = 0

#grab the paths to the input images in our dataset, then initialize
# out data list (which we'll soon populate)

#imagePaths = list(paths.list_images("./face-clustering/friendsvideo"))
#imagePaths = list(paths.list_images(args["dataset"]))
data = []

#print(imagePaths)


detector = cv2.dnn.readNetFromCaffe(protoPath, modelPath)
embedder = cv2.dnn.readNetFromTorch("openface_nn4.small2.v1.t7")


# loop over the image paths
total=0
knownEmbeddings=[]
knownNames=[]
for (i, imagePath) in enumerate(imagePaths):
	# extract the person name from the image path
	print("[INFO] processing image {}/{}".format(i + 1,
		len(imagePaths)))
	name = imagePath.split(os.path.sep)[-2]

	# load the image, resize it to have a width of 600 pixels (while
	# maintaining the aspect ratio), and then grab the image
	# dimensions
	image = cv2.imread(imagePath)
	image = imutils.resize(image, width=600)
	(h, w) = image.shape[:2]

	# construct a blob from the image
	imageBlob = cv2.dnn.blobFromImage(
		cv2.resize(image, (300, 300)), 1.0, (300, 300),
		(104.0, 177.0, 123.0), swapRB=False, crop=False)

	# apply OpenCV's deep learning-based face detector to localize
	# faces in the input image
	detector.setInput(imageBlob)
	detections = detector.forward()

	# ensure at least one face was found
	if len(detections) > 0:
		# we're making the assumption that each image has only ONE
		# face, so find the bounding box with the largest probability
		i = np.argmax(detections[0, 0, :, 2])
		confidence = detections[0, 0, i, 2]

		# ensure that the detection with the largest probability also
		# means our minimum probability test (thus helping filter out
		# weak detections)
		if confidence > 0.5:
			# compute the (x, y)-coordinates of the bounding box for
			# the face
			box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
			(startX, startY, endX, endY) = box.astype("int")

			# extract the face ROI and grab the ROI dimensions
			face = image[startY:endY, startX:endX]
			(fH, fW) = face.shape[:2]

			# ensure the face width and height are sufficiently large
			if fW < 20 or fH < 20:
				continue

			# construct a blob for the face ROI, then pass the blob
			# through our face embedding model to obtain the 128-d
			# quantification of the face
			faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255,
				(96, 96), (0, 0, 0), swapRB=True, crop=False)
			embedder.setInput(faceBlob)
			vec = embedder.forward()

			# add the name of the person + corresponding face
			# embedding to their respective lists
			knownNames.append(name)
			knownEmbeddings.append(vec.flatten())
	total += 1
# dump the facial encodings data to disk

# # dump the facial embeddings + names to disk
# print("[INFO] serializing {} encodings...".format(total))

# dump the facial encodings + names to disk
print("[INFO] serializing {} encodings...".format(total))
data = {"embeddings": knownEmbeddings, "names": knownNames}
f = open("./encodingsclass", "wb")
f.write(pickle.dumps(data))
f.close()

[INFO] quantifying faces...
[INFO] serializing 0 encodings...


In [4]:
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
import argparse
import pickle
import imutils

# load the face embeddings
print("[INFO] loading face embeddings...")
data = pickle.loads(open("./encodingsclass", "rb").read())
 
# encode the labels
print("[INFO] encoding labels...")
le = LabelEncoder()
labels = le.fit_transform(data["names"])
# train the model used to accept the 128-d embeddings of the face and
# then produce the actual face recognition
print("[INFO] training model...")
recognizer = SVC(C=1.0, kernel="linear", probability=True)
recognizer.fit(data["embeddings"], labels)
# write the actual face recognition model to disk
f = open("recognize", "wb")
f.write(pickle.dumps(recognizer))
f.close()
 
# write the label encoder to disk
f = open("lee", "wb")
f.write(pickle.dumps(le))
f.close()

[INFO] loading face embeddings...
[INFO] encoding labels...
[INFO] training model...


ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [5]:
# FACE classification using SVM classifier
# python recognize_faces_image.py --encodings encodings.pickle --image examples/example_01.png 

# import the necessary packages
import face_recognition
import argparse
import pickle
import imutils
import cv2
import os
import os.path
dir="./images/"
protoPath =  "./face_detection_model/deploy.prototxt"
modelPath = "./face_detection_model/res10_300x300_ssd_iter_140000.caffemodel"

detector = cv2.dnn.readNetFromCaffe(protoPath, modelPath)
embedder = cv2.dnn.readNetFromTorch("openface_nn4.small2.v1.t7")


# load the actual face recognition model along with the label encoder
recognizer = pickle.loads(open("recognize", "rb").read())
le = pickle.loads(open("lee", "rb").read())



# # load the known faces and embeddings
# print("[INFO] loading encodings...")
# data = pickle.loads(open("./encodingsclass", "rb").read())

# load the input image and convert it from BGR to RGB
imgname=input("Enter person name")
filename= dir+imgname+".jpg"
if os.path.isfile(filename)!=True:
    print("Person name doesn't exists")
else:
    image = cv2.imread(filename)
    image = imutils.resize(image, width=600)
    (h, w) = image.shape[:2]
    # construct a blob from the image
    imageBlob = cv2.dnn.blobFromImage(
        cv2.resize(image, (300, 300)), 1.0, (300, 300),
        (104.0, 177.0, 123.0), swapRB=False, crop=False)

    # apply OpenCV's deep learning-based face detector to localize
    # faces in the input image
    detector.setInput(imageBlob)
    detections = detector.forward()
    # loop over the detections
    for i in range(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with the
        # prediction
        confidence = detections[0, 0, i, 2]

        # filter out weak detections
        if confidence > 0.5:
            # compute the (x, y)-coordinates of the bounding box for the
            # face
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # extract the face ROI
            face = image[startY:endY, startX:endX]
            (fH, fW) = face.shape[:2]

            # ensure the face width and height are sufficiently large
            if fW < 20 or fH < 20:
                continue


            # construct a blob for the face ROI, then pass the blob
            # through our face embedding model to obtain the 128-d
            # quantification of the face
            faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255, (96, 96),
                (0, 0, 0), swapRB=True, crop=False)
            embedder.setInput(faceBlob)
            vec = embedder.forward()

            # perform classification to recognize the face
            preds = recognizer.predict_proba(vec)[0]
            j = np.argmax(preds)
            proba = preds[j]
            name = le.classes_[j]

            # draw the bounding box of the face along with the associated
            # probability
            text = "{}: {:.2f}%".format(name, proba * 100)
            y = startY - 10 if startY - 10 > 10 else startY + 10
            cv2.rectangle(image, (startX, startY), (endX, endY),
                (0, 0, 255), 2)
            cv2.putText(image, text, (startX, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)

    # show the output image
    cv2.imshow("Image", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'face_recognition'

In [ ]:
# FACE DETECTION RECOGNITION USING DLIB WITH COMPARISON IN VIDEO FILE

# import the necessary packages
import face_recognition
import argparse
import imutils
import pickle
import time
import cv2
from imutils.video import FPS

protoPath =  "./face_detection_model/deploy.prototxt"
modelPath = "./face_detection_model/res10_300x300_ssd_iter_140000.caffemodel"

detector = cv2.dnn.readNetFromCaffe(protoPath, modelPath)
embedder = cv2.dnn.readNetFromTorch("openface_nn4.small2.v1.t7")

# load the face embeddings
print("[INFO] loading face embeddings...")
data = pickle.loads(open("./encodingsclass", "rb").read())
 
# encode the labels
print("[INFO] encoding labels...")
le = LabelEncoder()
labels = le.fit_transform(data["names"])
# train the model used to accept the 128-d embeddings of the face and
# then produce the actual face recognition
print("[INFO] training model...")
recognizer = SVC(C=1.0, kernel="linear", probability=True)
recognizer.fit(data["embeddings"], labels)
# write the actual face recognition model to disk
f = open("recognize", "wb")
f.write(pickle.dumps(recognizer))
f.close()
 
# write the label encoder to disk
f = open("lee", "wb")
f.write(pickle.dumps(le))
f.close()


# load the actual face recognition model along with the label encoder
recognizer = pickle.loads(open("recognize", "rb").read())
le = pickle.loads(open("lee", "rb").read())

# # load the known faces and embeddings
# print("[INFO] loading encodings...")
# data = pickle.loads(open("./encodingsclass", "rb").read())

# initialize the pointer to the video file and the video writer
print("[INFO] processing video...")
stream = cv2.VideoCapture("adrian.mp4")
fps = FPS().start()
writer = None
display=1
# loop over frames from the video file stream
while True:
	# grab the next frame
	(grabbed, frame) = stream.read()

	# if the frame was not grabbed, then we have reached the
	# end of the stream
	if not grabbed:
		break
    
	image = imutils.resize(frame, width=600)
	(h, w) = image.shape[:2]
	r = frame.shape[1] / float(image.shape[1])
    
    # construct a blob from the image
	imageBlob = cv2.dnn.blobFromImage(
		cv2.resize(image, (300, 300)), 1.0, (300, 300),
		(104.0, 177.0, 123.0), swapRB=False, crop=False)

    # apply OpenCV's deep learning-based face detector to localize
    # faces in the input image
	detector.setInput(imageBlob)
	detections = detector.forward()
    # loop over the detections
	for i in range(0, detections.shape[2]):
		# extract the confidence (i.e., probability) associated with the
		# prediction
		confidence = detections[0, 0, i, 2]

		# filter out weak detections
		if confidence > 0.5:
			# compute the (x, y)-coordinates of the bounding box for the
			## face
			box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
			(startX, startY, endX, endY) = box.astype("int")

			## extract the face ROI
			face = image[startY:endY, startX:endX]
			(fH, fW) = face.shape[:2]

            # ensure the face width and height are sufficiently large
			if fW < 20 or fH < 20:
				continue


			# construct a blob for the face ROI, then pass the blob
			# through our face embedding model to obtain the 128-d
			# quantification of the face
			faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255, (96, 96),
				(0, 0, 0), swapRB=True, crop=False)
			embedder.setInput(faceBlob)
			vec = embedder.forward()

            # perform classification to recognize the face
			preds = recognizer.predict_proba(vec)[0]
			j = np.argmax(preds)
			proba = preds[j]
			name = le.classes_[j]

           # draw the bounding box of the face along with the associated
            # probability
			text = "{}: {:.2f}%".format(name, proba * 100)
			y = startY - 10 if startY - 10 > 10 else startY + 10
			cv2.rectangle(frame, (startX, startY), (endX, endY),
				(0, 0, 255), 2)
			cv2.putText(frame, text, (startX, y),
				cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)

    # show the output image
# 	cv2.imshow("Image", image)
# 	cv2.waitKey(0)
 
	if display > 0:
		cv2.imshow("Frame", frame)
		fps.update()
		key = cv2.waitKey(1) & 0xFF

		# if the `q` key was pressed, break from the loop
		if key == ord("q"):
			break

# stop the timer and display FPS information
fps.stop()
print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))
 
# do a bit of cleanup
stream.release()
cv2.destroyAllWindows()

# check to see if the video writer point needs to be released
if writer is not None:
	writer.release()

In [ ]:
# Face matching in video
# python recognize_faces_video.py --encodings encodings.pickle
# python recognize_faces_video.py --encodings encodings.pickle --output output/jurassic_park_trailer_output.avi --display 0

# import the necessary packages
from imutils.video import VideoStream
from imutils import paths
from os import listdir
from os.path import isfile, join
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
import face_recognition
import dlib
import argparse
import imutils
import pickle
import time
import cv2
import os
import numpy as np

protoPath =  "./face_detection_model/deploy.prototxt"
modelPath = "./face_detection_model/res10_300x300_ssd_iter_140000.caffemodel"

detector = cv2.dnn.readNetFromCaffe(protoPath, modelPath)
embedder = cv2.dnn.readNetFromTorch("openface_nn4.small2.v1.t7")

# load the face embeddings
print("[INFO] loading face embeddings...")
data = pickle.loads(open("./encodingsclass", "rb").read())
 
# encode the labels
print("[INFO] encoding labels...")
le = LabelEncoder()
labels = le.fit_transform(data["names"])
# train the model used to accept the 128-d embeddings of the face and
# then produce the actual face recognition
print("[INFO] training model...")
recognizer = SVC(C=1.0, kernel="linear", probability=True)
recognizer.fit(data["embeddings"], labels)
# write the actual face recognition model to disk
f = open("recognize", "wb")
f.write(pickle.dumps(recognizer))
f.close()
 
# write the label encoder to disk
f = open("lee", "wb")
f.write(pickle.dumps(le))
f.close()


# load the actual face recognition model along with the label encoder
recognizer = pickle.loads(open("recognize", "rb").read())
le = pickle.loads(open("lee", "rb").read())

# # load the known faces and embeddings
# print("[INFO] loading encodings...")
# data = pickle.loads(open("./encodingsclass", "rb").read())


# # load the known faces and embeddings
# print("[INFO] loading encodings...")
# data = pickle.loads(open("./encodingsclass", "rb").read())

# initialize the video stream and pointer to output video file, then
# allow the camera sensor to warm up
print("[INFO] starting video stream...")
_videodir = "./dataset"
#vs = cv2.VideoCapture(0)
vs = VideoStream(src=0).start()
writer = None
time.sleep(2.0)
display=1
count=0
saveimage='n'

# unknownfolder="./dataset/unknown1"
# if os.path.exists(unknownfolder)==True:
# #Remove images in temporary unknown1 folder                 
# 	filelist = [ f for f in os.listdir(unknownfolder) if f.endswith(".jpg") ]
# 	for f in filelist:
# 		os.remove(os.path.join(_unknownfolder, f))    
# # loop over frames from the video file stream
while True:
	# grab the frame from the threaded video stream
#	(grabbed, frame) = vs.read()
	frame = vs.read()
	# if the frame was not grabbed, then we have reached the
	# end of the stream
# 	if not grabbed:
# 		break
    
	image = imutils.resize(frame, width=600)
	(h, w) = image.shape[:2]
	r = frame.shape[1] / float(image.shape[1])
    
    # construct a blob from the image
	imageBlob = cv2.dnn.blobFromImage(
		cv2.resize(image, (300, 300)), 1.0, (300, 300),
		(104.0, 177.0, 123.0), swapRB=False, crop=False)

    # apply OpenCV's deep learning-based face detector to localize
    # faces in the input image
	detector.setInput(imageBlob)
	detections = detector.forward()
    # loop over the detections
	for i in range(0, detections.shape[2]):
		# extract the confidence (i.e., probability) associated with the
		# prediction
		confidence = detections[0, 0, i, 2]

		# filter out weak detections
		if confidence > 0.5:
			# compute the (x, y)-coordinates of the bounding box for the
			## face
			box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
			(startX, startY, endX, endY) = box.astype("int")

			## extract the face ROI
			face = image[startY:endY, startX:endX]
			(fH, fW) = face.shape[:2]

            # ensure the face width and height are sufficiently large
			if fW < 20 or fH < 20:
				continue


			# construct a blob for the face ROI, then pass the blob
			# through our face embedding model to obtain the 128-d
			# quantification of the face
			faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255, (96, 96),
				(0, 0, 0), swapRB=True, crop=False)
			embedder.setInput(faceBlob)
			vec = embedder.forward()

            # perform classification to recognize the face
			preds = recognizer.predict_proba(vec)[0]
			j = np.argmax(preds)
			proba = preds[j]
			name = le.classes_[j]

           # draw the bounding box of the face along with the associated
            # probability
			text = "{}: {:.2f}%".format(name, proba * 100)
			y = startY - 10 if startY - 10 > 10 else startY + 10
			cv2.rectangle(frame, (startX, startY), (endX, endY),
				(0, 0, 255), 2)
			cv2.putText(frame, text, (startX, y),
				cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
            
            
            
# 	# convert the input frame from BGR to RGB then resize it to have
# 	# a width of 750px (to speedup processing)
# 	rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
# 	rgb = imutils.resize(frame, width=750)
# 	r = frame.shape[1] / float(rgb.shape[1])

# 	# detect the (x, y)-coordinates of the bounding boxes
# 	# corresponding to each face in the input frame, then compute
# 	# the facial embeddings for each face
# 	boxes = face_recognition.face_locations(rgb, model="hog")
# 	encodings = face_recognition.face_encodings(rgb, boxes)
# 	names = []

# 	# loop over the facial embeddings
# 	for encoding in encodings:
# 		# attempt to match each face in the input image to our known
# 		# encodings
# 		matches = face_recognition.compare_faces(data["encodings"],
# 			encoding)
# 		name = "Unknown"

# 		# check to see if we have found a match
# 		if True in matches:
# 			# find the indexes of all matched faces then initialize a
# 			# dictionary to count the total number of times each face
# 			# was matched
# 			matchedIdxs = [i for (i, b) in enumerate(matches) if b]
# 			counts = {}

# 			# loop over the matched indexes and maintain a count for
# 			# each recognized face face
# 			for i in matchedIdxs:
# 				name = data["names"][i]
# 				counts[name] = counts.get(name, 0) + 1

# 			# determine the recognized face with the largest number
# 			# of votes (note: in the event of an unlikely tie Python
# 			# will select first entry in the dictionary)
# 			name = max(counts, key=counts.get)
		
# 		# update the list of names
# 		names.append(name)

# 	# loop over the recognized faces
# 	for ((top, right, bottom, left), name) in zip(boxes, names):
# 		# rescale the face coordinates
# 		top = int(top * r)
# 		right = int(right * r)
# 		bottom = int(bottom * r)
# 		left = int(left * r)

# 		# draw the predicted face name on the image
# 		cv2.rectangle(frame, (left, top), (right, bottom),
# 			(0, 255, 0), 2)
# 		y = top - 15 if top - 15 > 15 else top + 15
# 		cv2.putText(frame, name , (left, y), cv2.FONT_HERSHEY_SIMPLEX,
# 			0.75, (0, 255, 0), 2)
			cv2.imwrite("./dataset/unknown1/frame%d.jpg" %count,frame)
			count+=1
# 	# if the video writer is None *AND* we are supposed to write
# 	# the output video to disk initialize the writer
# 	if writer is None and output is not None:
# 		fourcc = cv2.VideoWriter_fourcc(*"MJPG")
# 		writer = cv2.VideoWriter(output, fourcc, 20,
# 			(frame.shape[1], frame.shape[0]), True)

# 	# if the writer is not None, write the frame with recognized
# 	# faces t odisk
# 	if writer is not None:
#  		writer.write(frame)

# # 	# check to see if we are supposed to display the output frame to
# # 	# the screen
	if display > 0:
		cv2.imshow("Frame", frame)
		
		key = cv2.waitKey(1) & 0xFF

		# if the `q` key was pressed, break from the loop
		if key == ord("q"):
			break

# do a bit of cleanup
# do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()

# check to see if the video writer point needs to be released
if writer is not None:
	writer.release()
    

#store new person


# from imutils.video import VideoStream
# import face_recognition
# import argparse
# import imutils
# import dlib
# import pickle
# import time
# import cv2

# AUTHENTICATE PERSON

_videodir= "./dataset/"
_videodir1 = "./dataset/unknown1/"
if name=='unknown':
	saveimage=input("Do you want to Authenticate this person ( Press y/n)?")
	if saveimage=='y':
		personname=input("Enter person name ")                
            
# create dynamic name, like "D:\Current Download\Attachment82673"
		_videodir2= os.path.join(_videodir, personname)

        # create 'dynamic' dir, if it does not exist
		if not os.path.exists(_videodir2):
			os.makedirs(_videodir2)
		listing = os.listdir(_videodir1)    
		for file in listing:
			img = cv2.imread(_videodir1+file)
			outfile= _videodir2+"/"+file
			cv2.imwrite(outfile,img)

#Remove images in temporary unknown1 folder
                            

		filelist = [ f for f in os.listdir(_videodir1) if f.endswith(".jpg") ]
		for f in filelist:
			os.remove(os.path.join(_videodir1, f))

		#_videodir2= "./dataset/tamilarasi"

#store face encodings of new person


# loop over the image paths
		total1=0
		knownEmbeddings1=[]
		knownNames1=[]
		imagePaths = list(paths.list_images(_videodir2))

		for (i, imagePath) in enumerate(imagePaths):
			# extract the person name from the image path
			print("[INFO] processing image {}/{}".format(i + 1,
				len(imagePaths)))
			name = imagePath.split(os.path.sep)[-2]

			# load the image, resize it to have a width of 600 pixels (while
			# maintaining the aspect ratio), and then grab the image
			# dimensions
            
			image = cv2.imread(imagePath)
			image = imutils.resize(image, width=600)
			(h, w) = image.shape[:2]

			# construct a blob from the image
			imageBlob = cv2.dnn.blobFromImage(
				cv2.resize(image, (300, 300)), 1.0, (300, 300),
				(104.0, 177.0, 123.0), swapRB=False, crop=False)

			# apply OpenCV's deep learning-based face detector to localize
			# faces in the input image
			detector.setInput(imageBlob)
			detections = detector.forward()

			# ensure at least one face was found
			if len(detections) > 0:
			# we're making the assumption that each image has only ONE
			# face, so find the bounding box with the largest probability
				i = np.argmax(detections[0, 0, :, 2])
				confidence = detections[0, 0, i, 2]

			# ensure that the detection with the largest probability also
			# means our minimum probability test (thus helping filter out
			# weak detections)
				if confidence > 0.5:
			# compute the (x, y)-coordinates of the bounding box for
			# the face
					box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
					(startX, startY, endX, endY) = box.astype("int")

			# extract the face ROI and grab the ROI dimensions
					face = image[startY:endY, startX:endX]
					(fH, fW) = face.shape[:2]

				# ensure the face width and height are sufficiently large
					if fW < 20 or fH < 20:
						continue

			# construct a blob for the face ROI, then pass the blob
			# through our face embedding model to obtain the 128-d
			# quantification of the face
					faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255,
						(96, 96), (0, 0, 0), swapRB=True, crop=False)
					embedder.setInput(faceBlob)
					vec = embedder.forward()

					# add the name of the person + corresponding face
					# embedding to their respective lists
					knownNames1.append(name)
					knownEmbeddings1.append(vec.flatten())
                
# #			image = cv2.imread(imagePath)
# 			rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# 			 # detect the (x, y)-coordinates of the bounding boxes
# 			# corresponding to each face in the input image
# 			boxes = face_recognition.face_locations(rgb,model="hog")
# 			# compute the facial embedding for the face
# 			encodings = face_recognition.face_encodings(rgb, boxes)
 
# 			for encoding in encodings:
# 				# add each encoding + name to our set of known names and
# 				# encodings
# 				knownEncodings1.append(encoding)
# 				knownNames1.append(name)
			total1 += 1
    # build a dictionary of the image path, bounding box location,
	# and facial encodings for the current image

	
# dump the facial encodings data to disk


# dump the facial encodings + names of new person to disk
		print("[INFO] loading new encodings...")
		data = pickle.loads(open("./encodingsclass", "rb").read())

		print("[INFO] serializing {} encodings...".format(total1))
		data1 = {"embeddings": knownEmbeddings1, "names": knownNames1}
		f = open("./encodingsclass", "wb")
		data.update(data1)
		f.write(pickle.dumps(data))
		f.close()
elif (saveimage!='y'):
	print("Not Authenticated person. Raise notification")

In [ ]:
data = pickle.loads(open("./encodingsclass", "rb").read())

print(data)